In [50]:
url = 'https://gis.westmorelandcountypa.gov/arcgis/rest/services/Upgrade/Parcels_AS400_VIEW_Upgrade/MapServer/2/query?where=1+%3D+1&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=%7B%22xmin%22%3A-8849573.386745242%2C%22ymin%22%3A4891969.810252989%2C%22xmax%22%3A-8847127.401840117%2C%22ymax%22%3A4894415.795158112%7D&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=500&units=esriSRUnit_StatuteMile&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=true&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset={}&resultRecordCount={}&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson'
count_url = 'https://gis.westmorelandcountypa.gov/arcgis/rest/services/Upgrade/Parcels_AS400_VIEW_Upgrade/MapServer/2/query?where=TWNSHP_NAM+%3D+%27HEMPFIELD%27&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=%7B%22xmin%22%3A-8849573.386745242%2C%22ymin%22%3A4891969.810252989%2C%22xmax%22%3A-8847127.401840117%2C%22ymax%22%3A4894415.795158112%7D&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=500&units=esriSRUnit_StatuteMile&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=true&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=true&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=0&resultRecordCount=&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=json'

In [31]:
import requests
import json
import pandas as pd
import geopandas as gpd

In [8]:
batch_size = 2000
count = requests.get(count_url).json()['count']

In [9]:
count

20933

In [10]:
out = requests.get(url.format(0)).json()

In [16]:
out.keys()

dict_keys(['type', 'features', 'exceededTransferLimit', 'properties'])

In [51]:
gj_out = None
loops = int(round(count / batch_size + 0.499999, 0))
print(f'Found {count} features. Doing {loops} loops')
last_record_limit =  count - batch_size * (loops - 1)

for i in range(loops):
    print(f'Loop {i}')
    if i == loops - 1:
        print(f'Last loop i={i}; loops={loops}.  Fetching {last_record_limit} records.')
        gj = requests.get(url.format(i*batch_size, last_record_limit)).json()
    else:
        gj = requests.get(url.format(i*batch_size, batch_size)).json()
    if gj_out is None:
        gj_out = gj
    else:
        gj_out['features'] += gj['features']
    
print(f'Found {len(gj_out['features'])} records.')
with open(f'gjout.geojson', 'w') as f:
    f.write(json.dumps(gj_out))

df = gpd.read_file('gjout.geojson')

df.to_csv('gjout.csv')

print(len(gj_out['features']))


Found 20933 features. Doing 11 loops
Loop 0
Loop 1
Loop 2
Loop 3
Loop 4
Loop 5
Loop 6
Loop 7
Loop 8
Loop 9
Loop 10
Last loop i=10; loops=11.  Fetching 933 records.
Found 20933 records.
20933


In [46]:
count - batch_size * (loops - 1)

933

In [58]:

df = gpd.read_file('gjout.geojson')


In [62]:
dfcsv = df.drop(['geometry'], axis='columns')
dfcsv.to_csv('gjout.csv')